In [ ]:
# default_exp model

# Model

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import torch

import pytorch_lightning as pl

from transformers import AdamW, EncoderDecoderModel, AutoTokenizer

In [ ]:
# export
class RetroFitModel(pl.LightningModule):
    def __init__(
        self,
        encoder_name,
        decoder_name,
        column,
        weight_decay=0.1,
        lr=5e-4,
        freeze_decoder=True
    ):
        super().__init__()
        self.encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_name)#, model_max_length=512)
        self.decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_name)#, model_max_length=512)
        self.decoder_tokenizer.pad_token = self.decoder_tokenizer.eos_token
        self.column = column
        self.weight_decay = weight_decay
        self.lr = lr
        self.freeze_decoder = freeze_decoder
        self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_name, decoder_name)
        self.model.config.decoder_start_token_id = self.decoder_tokenizer.bos_token_id
        self.model.config.pad_token_id = self.decoder_tokenizer.pad_token_id
        # self.model.config.vocab_size = self.model.config.decoder.vocab_size

    def training_step(self, batch, batch_idx):
        # retrieved = batch["retrieved_examples"]
        # retrieved = [self.encoder_tokenizer.eos_token.join(r) for r in retrieved]
        # input_ids = self.encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        # labels = self.decoder_tokenizer(batch[self.column], return_tensors="pt").input_ids
        # ["input_ids", "retrieved_input_ids", "attention_mask", "retrieved_attention_mask"]
        loss = self.model(input_ids=batch["retrieved_input_ids"], labels=batch["input_ids"]).loss

        self.log("trn_loss", loss, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def get_grouped_params(self, model, no_decay=["bias", "LayerNorm.weight"]):
        params_with_wd, params_without_wd = [], []
        for n, p in model.named_parameters():
            if any(nd in n for nd in no_decay):
                params_without_wd.append(p)
            else:
                params_with_wd.append(p)
        return [
            {"params": params_with_wd, "weight_decay": self.weight_decay},
            {"params": params_without_wd, "weight_decay": 0.0},
        ]

    def configure_optimizers(self):
        # Prepare the optimizer and learning rate scheduler
        # param_model = self.model# if not self.freeze_decoder else self.model.encoder
        # optimizer = AdamW(self.get_grouped_params(param_model), lr=self.lr)
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def generate(self, retrieved_examples, skip_special_tokens=True):
        retrieved = [self.encoder_tokenizer.eos_token.join(r) for r in retrieved_examples]
        input_ids = self.encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        generated = self.model.generate(input_ids, decoder_start_token_id=self.model.config.decoder.bos_token_id)

        return self.decoder_tokenizer.decode(generated, skip_special_tokens=skip_special_tokens)

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.ipynb.
Converted 01_model.ipynb.
Converted 02_tutorial.ipynb.
Converted index.ipynb.
